In [1]:
import os
import numpy as np
import pandas as pd
from pandas.core.algorithms import duplicated

from main import *
from parseSources import add_source_year_ids
from utilities import *
from parseSources import *

In [21]:
source_data = get_data_from_sources()
source_data = add_event_ids(source_data)
source_data = add_source_year_ids(source_data)

source_data = source_data.drop(columns="Sheet Names").drop_duplicates()
source_data.to_csv("source_data.csv", index=False)

In [28]:
sources_key = pd.read_csv("source keys/key_for_sources.csv")
sources_key

,Page #,2nd Page #,Premiums,Description,unique source ID
0,1,7.0,premiums,description,"1894-10-12 00:00:00,lewiston evening journal,1..."
1,2,NaN,premiums,NaN,"1881-10-11 00:00:00,oxford democrat,1881,oxfor..."


In [30]:
merged = source_data.merge(sources_key, how="left", on="unique source ID", suffixes=("_old", "")).fillna(source_data).drop(columns=["Page #_old", "2nd Page #_old", "Description_old", 'Premiums_old']).drop_duplicates()

merged

,County,Year,Event,Location,Source,Date,Event ID,unique source ID,Page #,2nd Page #,Premiums,Description
0,piscataquis,1912,piscataquis central fair,foxcroft,piscataquis observer,1912-10-03,"1912,piscataquis central fair","1912-10-03 00:00:00,piscataquis observer,1912,...",3,,premiums,
1,piscataquis,1894,piscataquis central fair,foxcroft,piscataquis observer,1894-09-27,"1894,piscataquis central fair","1894-09-27 00:00:00,piscataquis observer,1894,...",2,,premiums,
2,piscataquis,1892,piscataquis central fair,foxcroft,piscataquis observer,1892-09-29,"1892,piscataquis central fair","1892-09-29 00:00:00,piscataquis observer,1892,...",3,,premiums,
3,piscataquis,1884,piscataquis central fair,foxcroft,piscataquis observer,1884-10-09,"1884,piscataquis central fair","1884-10-09 00:00:00,piscataquis observer,1884,...",3,,premiums,
4,piscataquis,1887,piscataquis central fair,,piscataquis observer,1887-09-29,"1887,piscataquis central fair","1887-09-29 00:00:00,piscataquis observer,1887,...",2,,,description
...,...,...,...,...,...,...,...,...,...,...,...,...
1279,waldo,1914,north waldo and penobscot fair,monroe,republican journal,1914-09-17,"1914,north waldo and penobscot fair","1914-09-17 00:00:00,republican journal,1914,no...",,,premiums,
1280,waldo,1915,north waldo and penobscot fair,monroe,republican journal,1915-09-16,"1915,north waldo and penobscot fair","1915-09-16 00:00:00,republican journal,1915,no...",1,,premiums,
1281,waldo,1911,north waldo and penobscot fair,monroe,republican journal,1911-09-28,"1911,north waldo and penobscot fair","1911-09-28 00:00:00,republican journal,1911,no...",,,premiums,
1282,waldo,1888,north waldo and penobscot fair,monroe,maine farmer,1888-09-27,"1888,north waldo and penobscot fair","1888-09-27 00:00:00,maine farmer,1888,north wa...",,,premiums,


In [41]:
merged["DUPS unique Source ID"] = mask_source_id = merged.duplicated(subset=['unique source ID'], keep="last")
merged.drop([merged[mask_source_id]])

ValueError: 3

In [22]:
merged[merged["unique source ID"] == "1881-10-11 00:00:00,oxford democrat,1881,oxford county fair"]

,County_old,Year_old,Event_old,Location_old,Source_old,Date_old,Event ID_old,unique source ID,DUPS unique Source ID_old,DUPS Event ID_old,...,Location,Source,Page #,2nd Page #,Premiums,Description,Date,Event ID,DUPS Event ID,DUPS unique Source ID
757,oxford,1881,oxford county fair,,oxford democrat,1881-10-11 00:00:00,"1881,oxford county fair","1881-10-11 00:00:00,oxford democrat,1881,oxfor...",True,True,...,,oxford democrat,2.0,,premiums,description,1881-10-11,"1881,oxford county fair",True,True
802,oxford,1881,oxford county fair,,oxford democrat,1881-10-11 00:00:00,"1881,oxford county fair","1881-10-11 00:00:00,oxford democrat,1881,oxfor...",True,True,...,,oxford democrat,2.0,,premiums,,1881-10-11,"1881,oxford county fair",True,True


In [10]:
source_df = source_data_clean.copy()
mask_source_id = source_df.duplicated(subset=['Source ID'], keep="last")
third_source = source_df[mask_source_id].drop(columns=["Location", "Date", "County", "Sheet Names", "Event", "Year", "Source", "Page #", "2nd Page #", "Premiums", "Description"]).rename(columns={"S2 Description" : "S3 Description", "S2 Source" : "S3 Source", "S2 Page #": "S3 Page #", "S2 2nd Page #": "S3 2nd Page #", 'S2 Premiums': "S3 Premiums"})
source_df = source_df[~mask_source_id].merge(third_source, on=["Source ID"], how='left')

NameError: name 'source_data_clean' is not defined

In [7]:
source_df["dups"] = mask_notes_and_prde =  source_df.duplicated(subset=['Source ID'], keep=False)
source_df[mask_notes_and_prde].to_csv("key_for_sources.csv", index=False)